In [1]:
from llvm_ml.torch import BasicBlockDataset
banned_ids = []
dataset = BasicBlockDataset("./data/ryzen3600_v16.cbuf", masked=True, banned_ids=banned_ids, prefilter=True)
print(f"Training with {len(dataset)} samples")

KeyboardInterrupt: 

In [2]:
from llvm_ml.torch.nn import MCBERT, MCNNConfig
from torch_geometric.loader import DataLoader
from lightning.pytorch.callbacks import ModelSummary, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import torch
import lightning.pytorch as pl

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

config = MCNNConfig(dataset.num_opcodes)
config.learning_rate = 3e-5
config.batch_size = 256
config.hidden_size = 128
config.embedding_size = 256
# config.reg_forward_expansion = 16
config.forward_expansion = 2

num_training = int(0.7 * len(dataset))
num_val = len(dataset) - num_training

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_training, num_val])
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=6, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, num_workers=6, drop_last=True)

model = MCBERT(config)

logger = TensorBoardLogger("runs", name="bert_pretrain")
logger.log_graph(model)
callbacks = [
    ModelSummary(max_depth=-1),
    LearningRateMonitor(),
]
trainer = pl.Trainer(max_epochs=50,
                     logger=logger,
                     precision='16-mixed',
                     callbacks=callbacks,
                     #fast_dev_run=True,
                     #overfit_batches=1,
                     #log_every_n_steps=1,
                     )
trainer.fit(model, train_loader, val_loader)


Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                       | Type               | Params
-------------------------------------------------------------------
0  | embedding                  | MCEmbedding        | 5.4 M 
1  | embedding.embedding    

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/alex/anaconda3/envs/cpu-uarch-prediction-py11/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [4]:
torch.save(model, "data/bert_ryzen3600.pt")